In [3]:
import numpy as np
import matplotlib.pyplot as plt
from RobotController import RobotController

from coppeliasim_zmqremoteapi_client import *

client = RemoteAPIClient()

# get a remote object:
sim = client.require('sim')


# Função para rotacionar em torno do eixo z
def Rz(theta):
    return np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])

def get_object_position(sim, object_name):
    return sim.getObjectPosition(sim.getObject(object_name), -1)

def get_object_orientation(sim, object_name):
    return sim.getObjectOrientation(sim.getObject(object_name), -1)
def transformacao(posA, oriA):
    # Criação da matriz de transformação homogênea
    RWA = Rz(oriA[2])

    TWA = np.array([[posA[0]], [posA[1]], [posA[2]]]) # Transforma a posição em um array

    # Adiciona uma linha [0, 0, 0, 1] à direita da matriz RWA
    HWA = np.column_stack((RWA, TWA))
    HWA = np.row_stack((HWA, [0, 0, 0, 1]))  # Adiciona a linha [0, 0, 0, 1] ao final da matriz HWA

    return HWA

In [4]:
def readSensorData(range_data_signal_id="hokuyo_range_data", 
                    angle_data_signal_id="hokuyo_angle_data"):
    
    string_range_data = sim.getStringSignal(range_data_signal_id)

    string_angle_data = sim.getStringSignal(angle_data_signal_id)

    # verifique se ambos os dados foram obtidos corretamente
    if string_range_data != None and string_angle_data != None:
        # descompacte dados de mensagens de alcance e sensor
        raw_range_data = sim.unpackFloatTable(string_range_data)
        raw_angle_data = sim.unpackFloatTable(string_angle_data)

        return raw_range_data, raw_angle_data
    else:
        return np.array([]), np.array([])

def get_all_laser_data_ruido(laser_data, trans_global):
    dados_ruidos = []
    for ang, dist in laser_data:
        if dist < 5:  # Considerando leituras válidas do laser
            x = dist * np.cos(ang) + rd.uniform(-0.05, 0.05)  # Adicionando ruído na coordenada x
            y = dist * np.sin(ang) + rd.uniform(-0.05, 0.05)  # Adicionando ruído na coordenada y
            ponto_global = trans_global @ np.array([x, y, 1])
            dados_ruidos.append([ponto_global[0], ponto_global[1]])
    return dados_ruidos
    

In [8]:
# Parâmetros do grid de ocupação
map_size = [10, 10]  # Tamanho do mapa em metros
cell_size = 0.1  # Tamanho da célula em metros
rows, cols = int(map_size[0] / cell_size), int(map_size[1] / cell_size)  # Número de linhas e colunas

# Inicialização do occupancy grid
occupancy_grid = np.full((rows, cols), 0.5)  # Valor inicial 0.5 para representar células desconhecidas

# Calcula o log odds a partir da probabilidade
def log_odds(prob):
    return np.log(prob/(1-prob))

# Calcula a probabilidade a partir do log odds
def inv_log_odds(lo):
    return 1 - 1/(1+np.exp(lo))

# Atualiza a probabilidade de uma célula
def update_cell(prob, log_odds_val):
    return inv_log_odds(log_odds(prob) + log_odds_val)

# Cálculo do inverse sensor model adaptado para um laser
def new_inverse_range_sensor_model(cell, robot_cell, max_distance, l_0):
    x, y = cell
    x_robot, y_robot = robot_cell
    
    # Calcula a distância entre o robô e a célula
    r = np.sqrt((x - x_robot)**2 + (y - y_robot)**2)  # distância euclidiana
    
    max_dist = max_distance / cell_size
    alpha = 0.2 / cell_size
    distanceObs = np.linalg.norm(np.array(cell) - np.array(robot_cell[:2]))
    
    pocc = 0.85
    pfree = 0.1
    locc = log_odds(pocc)
    lfree = log_odds(pfree)

    if distanceObs > min(max_dist, max_distance + alpha / 2):  # Se a distância observada for maior que a distância máxima
        return log_odds(l_0)  # Retorna o log odds da célula anterior
    if max_distance < max_dist and abs(distanceObs - max_distance) <= alpha / 2:  # Se a distância observada for igual à distância máxima
        return locc  # Retorna o log odds da célula ocupada
    if distanceObs <= max_distance:  # Se a distância observada for menor que a distância máxima
        return lfree  # Retorna o log odds da célula livre

def bresenham(x0, y0, x1, y1):
    # Algoritmo de Bresenham para traçar linha
    cells = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy

    while True:
        cells.append((x0, y0))
        if x0 == x1 and y0 == y1:
            break
        e2 = 2 * err
        if e2 > -dy:
            err -= dy
            x0 += sx
        if e2 < dx:
            err += dx
            y0 += sy
    return cells

def new_occupancy_grid_mapping_test(occupancy_grid, robot_pos, robot_ori, laser_data):
    """
    Atualiza o occupancy grid com base nas leituras do laser.
    
    Parameters:
    - occupancy_grid: Grid de ocupação a ser atualizado
    - robot_pos: Posição do robô (x, y, z)
    - robot_ori: Orientação do robô (roll, pitch, yaw)
    - laser_data: Dados do laser [(distância, ângulo), ...]
    """
    l0 = 0.5  # Valor inicial de log odds

    robot_cell_x = int(robot_pos[0] / cell_size)
    robot_cell_y = int(robot_pos[1] / cell_size)
    robot_cell = (robot_cell_x, robot_cell_y)

    for distance, angle in laser_data:
        # Converte coordenadas polares para cartesianas
        x_laser = distance * np.cos(angle)
        y_laser = distance * np.sin(angle)

        # Rotaciona as coordenadas do laser para alinhar com a orientação do robô
        x_world = x_laser * np.cos(robot_ori[2]) - y_laser * np.sin(robot_ori[2])
        y_world = x_laser * np.sin(robot_ori[2]) + y_laser * np.cos(robot_ori[2])

        # Converte para coordenadas globais
        x_global = x_world + robot_pos[0]
        y_global = y_world + robot_pos[1]

        # Converte para células do grid
        cell_x = int(x_global / cell_size)
        cell_y = int(y_global / cell_size)
        laser_cell = (cell_x, cell_y)

        cells = bresenham(robot_cell[0], robot_cell[1], laser_cell[0], laser_cell[1])

        for (x, y) in cells:
            if 0 <= x < occupancy_grid.shape[0] and 0 <= y < occupancy_grid.shape[1]:
                lT_1 = occupancy_grid[x, y]
                l_cell = lT_1 + new_inverse_range_sensor_model([x, y], robot_cell, 10, lT_1) - log_odds(l0)
                occupancy_grid[x, y] = l_cell

def new_occupancy_grid_mapping(occupancy_grid, robot_cell, laser_cells):
    """
    Atualiza o occupancy grid com base nas leituras do laser.
    
    Parameters:
    - occupancy_grid: Grid de ocupação a ser atualizado
    - robot_cell: Célula do robô (x, y)
    - laser_cells: Células do laser [(x, y), ...]
    """
    for laser_cell in laser_cells:
        cells = bresenham(robot_cell[0], robot_cell[1], laser_cell[0], laser_cell[1])

        for i in range(len(cells)):
            cel_prob = occupancy_grid[cells[i][0], cells[i][1]]
            l_ant = log_odds(cel_prob)
            cel = np.array([cells[i][0], cells[i][1]])
            l_cell = l_ant + new_inverse_range_sensor_model(cel, robot_cell, 5, cel_prob)
            occupancy_grid[cells[i][0], cells[i],[1]] = inv_log_odds(l_cell)
    return occupancy_grid

def plot_occupancy_grid(occupancy_grid):
    # Função para plotar o grid de ocupação
    fig = plt.figure(figsize=(8, 8), dpi=100)
    ax = fig.add_subplot(111, aspect='equal')
    
    m = 1 - 1 / (1 + np.exp(occupancy_grid))
    
    plt.imshow(m, cmap='Greys', origin='upper', extent=(0, cols, rows, 0))
    plt.colorbar()
    ax.set_xticks(np.arange(0, cols, 1 / cell_size))
    ax.set_yticks(np.arange(0, rows, 1 / cell_size))
    plt.show()
    

# Exemplo de uso:
robot_pos = [2.0, 3.0, 0.0]  # Posição do robô (x, y, z)
robot_ori = [0.0, 0.0, np.deg2rad(30)]  # Orientação do robô (roll, pitch, yaw)
laser_data = [(1.0, np.deg2rad(0)), (1.5, np.deg2rad(45)), (2.0, np.deg2rad(90)), (8.0, np.deg2rad(-90))]  # Dados do laser [(distância, ângulo), ...]

#new_occupancy_grid_mapping_test(occupancy_grid, robot_pos, robot_ori, laser_data)


def get_laser_to_world(laser_data, laser_pos, laser_ori):
    # Função para calcular a matriz de transformação homogênea do laser para o mundo
    Hlw = transformacao(laser_pos, laser_ori)
    laser_to_world = []
    for i in range(len(laser_data)):
        ang, dist = laser_data[i] #pega os valores de angulo e distância
        x = dist * np.cos(ang) #meu x
        y = dist * np.sin(ang) #meu y
        point = np.array([x,y,0,1])
        point = Hlw @ point
        laser_to_world.append(point)
    return laser_to_world


# Conectando-se ao CoppeliaSim
clientID = sim.startSimulation()

if clientID != -1:
    print('Program Started')
    
    robot_controller = RobotController('kobuki')

    robot_controller.start_simulation()
    robot_pos = robot_controller.get_robot_position()
    robot_ori = robot_controller.get_robot_orientation()

    goal_position = [-2.4, -2.45, 0]  # Posição inicial do objetivo
    robot_controller.set_goal_position(goal_position)
    
    # Handle para os dados do LASER
    laser_range_data = "hokuyo_range_data"
    laser_angle_data = "hokuyo_angle_data"
    
    distance_goal = np.linalg.norm(np.array(goal_position[:2]) - np.array(robot_pos[:2]))
    print(f'Initial distance to goal: {distance_goal:.2f}')

    while distance_goal > 0.5:
        # Leitura do laser
        if laser_range_data is not None and laser_angle_data is not None:
            raw_range_data, raw_angle_data = readSensorData() 
            
        laser_data = np.array([raw_angle_data, raw_range_data]).T

        # Transformar dados do laser para o sistema de coordenadas global
        laser_pos = robot_controller.get_laser_position()
        laser_ori = robot_controller.get_laser_orientation()
        
        # Atualizando robo
        robot_pos = robot_controller.get_robot_position()
        robot_ori = robot_controller.get_robot_orientation()
        HWL = transformacao(robot_pos, robot_ori) 
        
        if laser_pos is not None and laser_ori is not None:
            laser_to_world = get_laser_to_world(laser_data, laser_pos, laser_ori)
        
        obs = []  # Lista de obstáculos
        obs_pts = []  # Lista de pontos dos obstáculos
    
       
            
        # Atualiza o occupancy grid
        new_occupancy_grid_mapping(occupancy_grid, robot_pos, obs)
        
        distance_goal = np.linalg.norm(np.array(goal_position[:2]) - np.array(robot_pos[:2]))
        print(f'Distance to goal: {distance_goal:.2f}')

    robot_controller.set_left_wheel_velocity(0)
    robot_controller.set_right_wheel_velocity(0)
    robot_controller.stop_simulation()
    print('Program ended')
    
    # Plotar o occupancy grid final
    plot_occupancy_grid(occupancy_grid)

Program Started
Initial distance to goal: 1.95


KeyboardInterrupt: 